<a href="https://colab.research.google.com/github/mdscolour/Learning/blob/main/MLzero4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 坤和的机器学期零基础笔记4

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


## Co-adaptation and dropout
maximum 50% for large NN, less than 20% for small NN

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(11) # should be 10, if 11 the last one will be negletcted
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2976 - accuracy: 0.9134
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1426 - accuracy: 0.9576
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1099 - accuracy: 0.9664
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0914 - accuracy: 0.9719
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0770 - accuracy: 0.9760


In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0706 - accuracy: 0.9787 - 423ms/epoch - 1ms/step


[0.0706235021352768, 0.9786999821662903]

In [ ]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [ ]:
print(model.predict(x_test[99].reshape(-1,28,28)))
print(np.argmax(model.predict(x_test[99].reshape(-1,28,28))))

1/1 [==============================] - 0s 15ms/step
[[ -7.6501765 -11.678655   -4.1637235   0.2439465  -1.0369728  -7.3296037
  -15.307793    2.3761663  -2.7242131  12.267004  -20.255722 ]]
1/1 [==============================] - 0s 19ms/step
9


In [ ]:
np.argmax(probability_model(x_test[99].reshape(-1,28,28)))

9

## batch 和自定义模型

In [ ]:
tdataset = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])

ds = tdataset.shuffle(3).batch(4)

count = 0
for example in ds.take(15):
  batch = example.numpy()
  print(count,batch)
  count+=1

0 [2 1 4 6]
1 [3 7 5 8]
2 [ 9 12 10 13]
3 [11]


In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(32, 3, activation='relu')
    self.flatten = tf.keras.layers.Flatten()
    self.d1 = tf.keras.layers.Dense(128, activation='relu')
    self.d2 = tf.keras.layers.Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)
    return x

# Create an instance of the model
model = MyModel()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 87s 46ms/step - loss: 0.0885 - accuracy: 0.9730
Epoch 2/2
1875/1875 [==============================] - 88s 47ms/step - loss: 0.0381 - accuracy: 0.9883


In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 3s - loss: 0.0601 - accuracy: 0.9805 - 3s/epoch - 11ms/step


[0.06014706566929817, 0.9804999828338623]

In [ ]:
model.fit(train_ds, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 81s 43ms/step - loss: 0.1337 - accuracy: 0.9592
Epoch 2/2
1875/1875 [==============================] - 81s 43ms/step - loss: 0.0401 - accuracy: 0.9872


In [ ]:
model.evaluate(test_ds, verbose=2)

313/313 - 4s - loss: 0.0569 - accuracy: 0.9811 - 4s/epoch - 14ms/step


[0.05690796673297882, 0.9811000227928162]